In [3]:
data_path = "/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/"

In [9]:
# imports
# import the necessary packages
import pandas as pd
import numpy as np
import random
random.seed(1)
import csv
import cv2
import os,re, codecs
import tensorflow.keras.metrics
from tqdm import tqdm
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from gensim.models import KeyedVectors
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.utils import to_categorical
from imutils import paths
from tensorflow.python.keras.layers.merge import Concatenate, Average, concatenate
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from numpy import asarray,zeros
import seaborn as sns
from sklearn.model_selection import KFold
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('always') 

/sda/rina_1921cs13/anaconda3/envs/newnisbert/lib/python3.6/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [15]:
#Lines to run the code on GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from tensorflow.python.keras import backend as K
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
K.set_session(tf.compat.v1.Session(config=config))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:18:00.0, compute capability: 7.5



In [19]:
#--------------------Image Input-------------------------------------------
train = pd.read_csv('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/data.csv')

image_id=train.image_id.values
image_id=image_id.tolist()
print('Total number of image ids', len(image_id))
unique_images = list(np.unique(image_id))
num_unique_image = len(unique_images)
print('Unique images', num_unique_image) # Some images are repeated and have the more than one emotion label

Total number of image ids 3045
Unique images 2123


In [24]:
# example of loading an image with the Keras API
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19,VGG16
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input

image_embed = []
item_lab = ["anger", "fear", "joy", "love", "sadness", "surprise"]
item = ["athletes","baby", "couple","cat","crowd","dog","family","girl","guy","horse","lion","soldier","teacher","teenager","tiger","people","scenery", "events", "park", "animals", "group"]

label =[]
for num, i in enumerate(item_lab):
    fol_dir = os.path.join('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/images/', i)
    sub_dirs = os.listdir(fol_dir)
    for j in sub_dirs:
        if os.path.isdir("/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/images/"+i+"/"+j):
            print(i+'/',j)
            for k in os.listdir("/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/images/"+i+"/"+j):
                img = load_img("/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/images/"+i+"/"+j+"/"+k,target_size=(224, 224))	
                img = img_to_array(img)
                img = preprocess_input(img)
                image_embed.append(img)
                label.append(num)
        else:
            print("path does not exist")

data = np.array(image_embed)
label_arr = np.array(label)
print(data.shape)

anger/ guy
anger/ dog
anger/ crowd
anger/ girl
anger/ horse
anger/ couple
anger/ baby
anger/ athletes
anger/ family
anger/ soldier
anger/ tiger
anger/ teenager
anger/ lion
anger/ teacher
anger/ cat
fear/ guy
fear/ dog
fear/ girl
fear/ couple
fear/ scenery
fear/ baby
fear/ family
fear/ cat
joy/ events
joy/ guy
joy/ dog
joy/ girl
joy/ scenery
joy/ baby
joy/ athletes
joy/ family
joy/ park
joy/ teenager
joy/ teacher
joy/ cat
love/ dog
love/ people
love/ couple
love/ scenery
love/ baby
love/ animals
love/ teenager
love/ cat
sadness/ guy
sadness/ girl
sadness/ people
sadness/ couple
sadness/ scenery
sadness/ baby
sadness/ park
sadness/ teenager
sadness/ group
surprise/ people
surprise/ couple
surprise/ scenery
surprise/ baby
surprise/ family
surprise/ cat
(3045, 224, 224, 3)


In [25]:
print('image data shape', data.shape)
print('labels shape', label_arr.shape)

image data shape (3045, 224, 224, 3)
labels shape (3045,)


In [26]:
# Saving the data
np.save('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/data.npy', data)
np.save('/sda/rina_1921cs13/Fake_Emo/UnBiasedEmo/label.npy', label_arr)